# 🛒 Análisis Exploratorio de Datos (EDA): E-Commerce Retail
- Autor: Erickson Otaño Data Engineer 
- Herramientas: PySpark, Spark SQL, Databricks Dashboard
- Link fuente de Datos: https://www.kaggle.com/datasets/carrie1/ecommerce-data/data

## 📋 Descripción del Proyecto
Este notebook documenta el ciclo completo de procesamiento y análisis de un dataset transaccional de E-Commerce (~540k registros). El objetivo es transformar datos crudos en insights de negocio accionables, simulando un entorno de producción Big Data.

## ⚙️ Flujo de Trabajo
- Ingesta y Calidad de Datos: Carga de datos y limpieza rigurosa utilizando PySpark. Se implementaron estrategias robustas para el manejo de nulos y la estandarización de formatos de fecha complejos.

- Ingeniería de Características: Creación de métricas temporales y financieras para enriquecer el dataset.

- Análisis Avanzado (SQL): Ejecución de consultas complejas utilizando Window Functions y CTEs para realizar:

- Segmentación de clientes RFM (Recencia, Frecuencia, Monetario).

- Detección de tendencias estacionales y outliers.

- Análisis de Pareto (Top Productos).

- Visualización Ejecutiva: Creación de Vistas Optimizadas y un Dashboard Interactivo final.

📍 **Nota de Navegación:** Este análisis culmina con un Dashboard Ejecutivo. Puede acceder a él cambiando la vista de este Notebook a "Dashboard" en el menú superior.

# FASE 1: CARGA Y EXPLORACIÓN INICIAL

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum as spark_sum 
import pyspark.sql.functions as F # Importar el alias F para literales y funciones

# RUTA DEL DATASET (ajustar si es necesario)
DATA_PATH = "/Volumes/eda_ecommerce/default/ecommerce_files/E-Commerce Data.csv" 

df_raw = spark.read.csv(
    DATA_PATH,
    header=True,
    inferSchema=True
)

print("=" * 80)
print("VALIDACIÓN INICIAL DE DATOS")
print("=" * 80)

## SECCIÓN 1.2: Mostrar Schema y estructura

In [0]:
print("\n📋 SCHEMA DEL DATASET:")
print("-" * 80)
df_raw.printSchema()

print("\n📊 DIMENSIONES:")
# El conteo puede tardar unos segundos en Databricks Free Edition
print(f"  • Registros: {df_raw.count():,}") 
print(f"  • Columnas: {len(df_raw.columns)}")

# ============================================================================
# SECCIÓN 1.3: Nulos y Estadísticas (Captura el estado inicial)
# ============================================================================

print("\n📊 INFORMACIÓN DE NULOS POR COLUMNA (Antes de Limpieza):")
print("-" * 80)
null_counts = df_raw.select(
    [spark_sum(col(c).isNull().cast("int")).alias(c) for c in df_raw.columns]
)
display(null_counts)

# FASE 2: LIMPIEZA Y PREPARACIÓN (Orden Optimizado)

Vamos a tomar la decisión de negocio de eliminar los nulos en CustomerID y Description, ya que el análisis RFM y de productos es central.

In [0]:
print("\n" + "=" * 80)
print("FASE 2: LIMPIEZA Y PREPARACIÓN (OPTIMIZADA)")
print("=" * 80)

print("\n🧹 2.0: LIMPIEZA DE NULOS Y DUPLICADOS (Optimizando rendimiento)")

# 1. Eliminar filas con CustomerID nulo (CRÍTICO para análisis RFM)
df_cleaned = df_raw.filter(col("CustomerID").isNotNull())

# 2. Eliminar filas con Description nula (Menor impacto, pero mejora calidad)
df_cleaned = df_cleaned.filter(col("Description").isNotNull())

# 3. Eliminar duplicados exactos
df_cleaned = df_cleaned.dropDuplicates()

# 4. Filtrar transacciones válidas (Quantity > 0 y UnitPrice > 0)
# Esto también elimina las devoluciones/cancelaciones (Quantity < 0), que analizaremos luego
df_cleaned = df_cleaned.filter(
    (col("Quantity") > 0) & (col("UnitPrice") > 0)
)

print(f"📊 Registros antes de limpieza: {df_raw.count():,}")
print(f"📊 Registros después de limpieza: {df_cleaned.count():,}")
print(f"📉 Pérdida total de registros: {df_raw.count() - df_cleaned.count():,}")
print("✅ Limpieza básica completada.")

## SECCIÓN 2.1: Conversión de tipos de datos (ERROR DE FORMATO CORREGIDO)

Ahora que el DataFrame es más pequeño, aplicamos la conversión de la fecha, utilizando la sintaxis correcta que corregimos antes (F.lit("MM/d/yyyy H:m")).

In [0]:
from pyspark.sql import functions as F

print("\n🔄 2.1: CONVERSIÓN DE InvoiceDate a TIMESTAMP (USANDO TRY)...")

# USAMOS try_to_timestamp + FORMATO FLEXIBLE (M/d/yyyy H:m)
# M: Acepta meses de 1 o 2 dígitos (1 y 12 funcionan)
# d: Acepta días de 1 o 2 dígitos (1 y 25 funcionan)
# try_to_timestamp: Si falla, pone NULL en vez de dar error.

df_cleaned = df_cleaned.withColumn(
    "InvoiceDate_Timestamp",
    F.coalesce(
        F.try_to_timestamp(F.col("InvoiceDate"), F.lit("M/d/yyyy H:mm")),
        F.try_to_timestamp(F.col("InvoiceDate"), F.lit("MM/dd/yyyy HH:mm"))
    )
).drop("InvoiceDate").withColumnRenamed("InvoiceDate_Timestamp", "InvoiceDate")

print("✅ InvoiceDate convertido a timestamp")

# Verificamos si quedaron nulos (fechas que no se pudieron leer)
failed_count = df_cleaned.filter(F.col("InvoiceDate").isNull()).count()

if failed_count > 0:
    print(f"⚠️ Advertencia: {failed_count} fechas no se pudieron convertir (son NULL).")
else:
    print("✨ Éxito Total: 0 errores de conversión.")

df_cleaned.printSchema()
display(df_cleaned.select("InvoiceDate").limit(5))

## SECCIÓN 2.2: Crear columnas derivadas

In [0]:
print("\n📝 2.2: CREANDO COLUMNAS DERIVADAS...")

df_cleaned = df_cleaned.withColumn(
    "Year", year(col("InvoiceDate"))
).withColumn(
    "Month", month(col("InvoiceDate"))
).withColumn(
    "Quarter", quarter(col("InvoiceDate"))
).withColumn(
    "DayOfWeek", dayofweek(col("InvoiceDate"))
).withColumn(
    "YearMonth",
    concat_ws("-", col("Year"), lpad(col("Month"), 2, "0")) # Formato AAAA-MM
).withColumn(
    "TransactionAmount",
    round(col("Quantity") * col("UnitPrice"), 2) # Revenue por línea
)

print("✅ Columnas derivadas creadas: Year, Month, YearMonth, TransactionAmount, etc.")
display(df_cleaned.select("InvoiceDate", "YearMonth", "TransactionAmount").limit(5))

## SECCIÓN 2.3: Crear vista temporal para SQL queries

In [0]:
print("\n📌 2.3: CREANDO VISTA TEMPORAL...")

df_cleaned.createOrReplaceTempView("sales_cleaned")
print("✅ Vista 'sales_cleaned' creada para Spark SQL")

# Validar vista
spark.sql("SELECT COUNT(*) as TotalRegistrosLimpio FROM sales_cleaned").show()

# FASE 3: ANÁLISIS CON SPARK SQL

## QUERY 1: Top 10 Países por Ingresos (Best Sellers)

In [0]:
print("\n" + "=" * 80)
print("FASE 3: ANÁLISIS CON SPARK SQL")
print("=" * 80)

# 1. Definimos la fecha en PYTHON (Más seguro que usar SET SQL)
today_date = '2011-12-10'
print(f"📌 Fecha de Corte establecida en Python: {today_date}")

# ----------------------------------------------------------------------------
# QUERY 1: Top 10 Países por Ingresos (Best Sellers)
# ----------------------------------------------------------------------------
print("\n🔍 QUERY 1: Top Países por Revenue y Ticket Promedio")

# Esta consulta no necesita la fecha, pero la ejecutamos para validar
df_q1 = spark.sql("""
SELECT 
    Country,
    COUNT(DISTINCT InvoiceNo) AS TotalInvoices,
    SUM(TransactionAmount) AS TotalRevenue,
    ROUND(AVG(TransactionAmount), 2) AS AvgTicket
FROM sales_cleaned
GROUP BY Country
ORDER BY TotalRevenue DESC
LIMIT 10
""")

display(df_q1)

## QUERY 2: Tendencia Mensual

In [0]:
print("\n🔍 QUERY 2: Tendencia Mensual de Ventas")

df_q2 = spark.sql("""
SELECT 
    YearMonth,
    SUM(TransactionAmount) AS MonthlyRevenue,
    COUNT(DISTINCT InvoiceNo) AS MonthlyTransactions
FROM sales_cleaned
GROUP BY YearMonth
ORDER BY YearMonth
""")

display(df_q2)

## QUERY AVANZADA (RFM) - Aquí usamos la variable de Python

In [0]:
print(f"\n🔍 QUERY 3 (Pre-RFM): Cálculo de Recencia usando fecha corte {today_date}")

# NOTA: Usamos f""" para inyectar la variable '{today_date}' dentro del SQL
df_rfm_preview = spark.sql(f"""
SELECT 
    CustomerID,
    MAX(InvoiceDate) as LastPurchase,
    -- Inyectamos la variable aquí abajo vvv
    DATEDIFF('{today_date}', MAX(InvoiceDate)) AS RecencyDays,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    SUM(TransactionAmount) AS Monetary
FROM sales_cleaned
GROUP BY CustomerID
ORDER BY Monetary DESC
LIMIT 5
""")

display(df_rfm_preview)

## QUERY 4: Distribución de Ventas por Hora del Día

In [0]:
print("\n🔍 QUERY 4: Distribución de Ventas por Hora del Día (Visualización de Barras)")

df_q4 = spark.sql("""
SELECT
    HOUR(InvoiceDate) AS PurchaseHour,
    COUNT(InvoiceNo) AS TotalTransactions,
    SUM(TransactionAmount) AS TotalRevenue
FROM sales_cleaned
GROUP BY PurchaseHour
ORDER BY PurchaseHour
""")

display(df_q4)
# Insight Clave: Buscar la hora pico de compra para optimizar servidores o promociones.

## QUERY 5: Top 10 Productos por Revenue (Líderes de Ingresos)

In [0]:
print("\n🔍 QUERY 5: Top 10 Productos por Revenue")

df_q5 = spark.sql("""
SELECT
    StockCode,
    Description,
    SUM(TransactionAmount) AS TotalRevenue,
    SUM(Quantity) AS TotalQuantity,
    COUNT(DISTINCT CustomerID) AS TotalCustomers
FROM sales_cleaned
GROUP BY StockCode, Description
ORDER BY TotalRevenue DESC
LIMIT 10
""")

display(df_q5)
# Insight Clave: ¿Los productos que más ingresos generan son los que más cantidad venden?

## QUERY 6: Identificación de Outliers (Facturas de Alto Valor)
Requiere GROUP BY y luego Window Functions (AVG OVER PARTITION)

In [0]:
print("\n🔍 QUERY 6: Identificación de Outliers (Facturas de Alto Valor)")

df_q6 = spark.sql("""
SELECT
    InvoiceNo,
    InvoiceDate,
    CustomerID,
    Country,
    SUM(TransactionAmount) AS InvoiceTotal,
    -- Comparamos la factura con el promedio de todas las facturas de su país
    ROUND(AVG(SUM(TransactionAmount)) OVER (PARTITION BY Country), 2) AS AvgCountryInvoice
FROM sales_cleaned
GROUP BY InvoiceNo, InvoiceDate, CustomerID, Country
ORDER BY InvoiceTotal DESC
LIMIT 5
""")

display(df_q6)

## QUERY 7: Segmentación RFM Completa (VIP, Leales, En Riesgo)

Uso de CTEs y NTILE (Window Function)

In [0]:
print(f"\n🔍 QUERY 7: Segmentación RFM Completa (Usando NTILE y CTE) - Fecha corte: {today_date}")

df_q7 = spark.sql(f"""
WITH rfm_base AS (
    -- 1. Calcular Recency, Frequency, Monetary
    SELECT
        CustomerID,
        DATEDIFF('{today_date}', MAX(InvoiceDate)) AS Recency,
        COUNT(DISTINCT InvoiceNo) AS Frequency,
        ROUND(SUM(TransactionAmount), 2) AS Monetary
    FROM sales_cleaned
    GROUP BY CustomerID
),
rfm_scores AS (
    -- 2. Asignar Scores (NTILE)
    SELECT
        CustomerID,
        Monetary,
        -- R_Score: Cuanto MENOS Recency, MEJOR Score (ORDER BY DESC para invertir el ranking)
        NTILE(4) OVER (ORDER BY Recency DESC) AS R_Score, 
        -- F_Score: Cuanto MÁS Frequency, MEJOR Score
        NTILE(4) OVER (ORDER BY Frequency ASC) AS F_Score,
        -- M_Score: Cuanto MÁS Monetary, MEJOR Score
        NTILE(4) OVER (ORDER BY Monetary ASC) AS M_Score
    FROM rfm_base
)
-- 3. Segmentación Final
SELECT 
    CASE 
        WHEN R_Score = 4 AND F_Score = 4 AND M_Score = 4 THEN 'A. VIP / Champions'
        WHEN (R_Score >= 3 AND F_Score >= 3) OR M_Score = 4 THEN 'B. Loyal Customers'
        WHEN R_Score = 1 AND F_Score <= 2 THEN 'D. At Risk / Churning'
        WHEN R_Score >= 3 AND F_Score <= 2 THEN 'C. New Customers'
        ELSE 'E. Other'
    END as Segment,
    COUNT(CustomerID) as CustomerCount,
    ROUND(AVG(Monetary), 2) as AvgMonetary,
    ROUND(SUM(Monetary), 2) as TotalRevenue
FROM rfm_scores
GROUP BY Segment
ORDER BY TotalRevenue DESC
""")

display(df_q7)

# FASE 4: VISUALIZACIONES ESTRATÉGICAS

## VISUALIZACIÓN 1: Tendencia de Ingresos Mensuales

Objetivo: Mostrar la estacionalidad y el crecimiento.


In [0]:
print("\n📊 VIZ 1: Tendencia Mensual (Configurar como Line Chart)")

df_viz1 = spark.sql("""
SELECT 
    YearMonth,
    SUM(TransactionAmount) AS Revenue
FROM sales_cleaned
GROUP BY YearMonth
ORDER BY YearMonth
""")

display(df_viz1)

Databricks visualization. Run in Databricks to view.

## Top 10 Productos por Ingresos
Objetivo: Identificar los "Best Sellers" (Principio de Pareto).

In [0]:
print("\n📊 VIZ 2: Top 10 Productos (Configurar como Bar Chart Horizontal)")

df_viz2 = spark.sql("""
SELECT 
    Description, -- Usamos la descripción para que sea legible en el gráfico
    SUM(TransactionAmount) AS TotalRevenue
FROM sales_cleaned
GROUP BY Description
ORDER BY TotalRevenue DESC
LIMIT 10
""")

display(df_viz2)

Databricks visualization. Run in Databricks to view.

## VISUALIZACIÓN 3: Heatmap de Ventas (Día de Semana vs Hora)

Objetivo: Encontrar el "momento relevantes" de la semana (Día vs Hora).


In [0]:
print("\n📊 VIZ 3: Intensidad de Ventas (Configurar como Heatmap)")

df_viz3 = spark.sql("""
SELECT 
    CASE 
        WHEN DayOfWeek = 1 THEN 'Dom'
        WHEN DayOfWeek = 2 THEN 'Lun'
        WHEN DayOfWeek = 3 THEN 'Mar'
        WHEN DayOfWeek = 4 THEN 'Mie'
        WHEN DayOfWeek = 5 THEN 'Jue'
        WHEN DayOfWeek = 6 THEN 'Vie'
        WHEN DayOfWeek = 7 THEN 'Sab'
    END AS DiaSemana,
    DayOfWeek, -- Para ordenar correctamente
    HOUR(InvoiceDate) AS HoraDia,
    SUM(TransactionAmount) AS Revenue
FROM sales_cleaned
GROUP BY DayOfWeek, HoraDia
ORDER BY DayOfWeek, HoraDia
""")

display(df_viz3)

Databricks visualization. Run in Databricks to view.

## VISUALIZACIÓN 4: Scatter Plot (Precio vs Cantidad)

Objetivo: Ver la elasticidad. ¿Los productos más baratos se venden en mayor volumen? Nota: Limitamos a 1000 puntos para no saturar el navegador.

In [0]:
print("\n📊 VIZ 4: Elasticidad de Precio (Configurar como Scatter)")

df_viz4 = spark.sql("""
SELECT 
    UnitPrice,
    Quantity,
    Country
FROM sales_cleaned
WHERE Quantity > 0 AND UnitPrice < 50 -- Filtramos outliers extremos para mejor visualización
LIMIT 1000
""")

display(df_viz4)

Databricks visualization. Run in Databricks to view.

# FASE 5: HALLAZGOS Y CONCLUSIONES

# 1. Resumen Ejecutivo
Este Análisis Exploratorio de Datos (EDA) sobre las transacciones de E-Commerce (Dic 2010 - Dic 2011) se realizó utilizando PySpark y Spark SQL en Databricks. La limpieza rigurosa eliminó el 27% de los registros (149,217 filas) para asegurar la integridad de las métricas clave, como la segmentación RFM.

El análisis concluye que el negocio opera con un alto **riesgo de concentración geográfica** y una fuerte **dependencia estacional** de las compras de fin de año. La identificación del segmento **VIP/Champions** (4-4-4) es crucial para enfocar las estrategias de retención.

---

## 2. Insights Clave (Basados en Fases 3 y 4)

| Insight | Dato Clave | Impacto de Negocio |
| :--- | :--- | :--- |
| **Riesgo Geográfico** | Reino Unido genera **~90%** del Revenue. | La dependencia de un único mercado supone un riesgo si las condiciones económicas locales cambian. |
| **Estacionalidad de Pico** | Noviembre es el mes con el **mayor Revenue** histórico. | El ciclo de planificación de inventario debe comenzar en Septiembre/Octubre para capitalizar este pico. |
| **Horas de Compra** | El 80% de las ventas ocurre entre **10:00 AM y 3:00 PM**. | La ventana de alta actividad es limitada. Las promociones y el despliegue de recursos deben centrarse en estas horas. |
| **Segmentación RFM** | El segmento **VIP/Champions** (clientes con alta Recencia, Frecuencia y Monetario) es el motor de valor. | Retener a este grupo es la prioridad número uno; es más económico que adquirir nuevos clientes. |
| **Anomalías (Outliers)** | Se identificaron transacciones con valores inusualmente altos (ej. > 10,000 USD), lo cual requiere una revisión manual de facturas específicas. | Potenciales errores de *data entry* o clientes B2B masivos que deben ser gestionados por un equipo de cuentas clave. |

---

## 3. Recomendaciones Accionables

1.  **Diversificación Geográfica:** Lanzar campañas promocionales específicas en **Alemania y Francia** (los siguientes países con mayor potencial de ingresos) en Q1 del próximo año, utilizando su promedio de *AvgTicket* como base para la segmentación de ofertas.
2.  **Optimización Operativa:** Programar el mantenimiento del sitio web, el despliegue de nuevas funcionalidades y las actualizaciones de inventario para las horas de baja actividad (ej. 1 AM - 7 AM) para maximizar la disponibilidad durante la ventana pico de ventas (10 AM - 3 PM).
3.  **Estrategia de Retención VIP:** Crear un canal de comunicación exclusivo y ofrecer beneficios (ej. *Early Access* a nuevos productos o envíos gratuitos prioritarios) a los clientes del segmento **VIP/Champions** identificado por el análisis RFM.

---

## 4. Limitaciones y Trabajo Futuro
* **Limitación de Datos:** El dataset no incluye el costo de los bienes vendidos (COGS), por lo que el análisis se basó en el Revenue bruto, sin poder calcular el margen de beneficio real.
* **Trabajo Futuro:** Integrar la dimensión de **costos** para realizar un análisis de **Profitability** por producto y cliente, y no solo de Revenue. Esto permitiría tomar decisiones de negocio basadas en la rentabilidad neta.

# FASE 6: PREPARACIÓN PARA DASHBOARD (CREACIÓN DE VIEWS)

In [0]:
print("\n" + "=" * 80)
print("FASE 6: CREACIÓN DE VIEWS OPTIMIZADAS")
print("=" * 80)

# Definimos la fecha de corte en Python
today_date = '2011-12-10'

# View 1: Métrica temporal (para el Line Chart)
spark.sql("""
CREATE OR REPLACE TEMP VIEW vw_monthly_revenue AS
SELECT 
    YearMonth,
    SUM(TransactionAmount) AS Revenue,
    COUNT(DISTINCT InvoiceNo) AS Transactions
FROM sales_cleaned
GROUP BY YearMonth
ORDER BY YearMonth
""")
print("✅ View 'vw_monthly_revenue' creada")

# View 2: Segmentación RFM (CORREGIDA)
# Se incluye 'Monetary' en el CTE rfm_scores para poder usarlo en el promedio final
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW vw_customer_segments AS
WITH rfm_base AS (
    SELECT
        CustomerID,
        DATEDIFF('{today_date}', MAX(InvoiceDate)) AS Recency,
        COUNT(DISTINCT InvoiceNo) AS Frequency,
        ROUND(SUM(TransactionAmount), 2) AS Monetary
    FROM sales_cleaned
    GROUP BY CustomerID
),
rfm_scores AS (
    SELECT
        CustomerID,
        Monetary, -- <--- ¡AQUÍ ESTABA EL ERROR! Faltaba pasar esta columna
        NTILE(4) OVER (ORDER BY Recency DESC) AS R_Score, 
        NTILE(4) OVER (ORDER BY Frequency ASC) AS F_Score,
        NTILE(4) OVER (ORDER BY Monetary ASC) AS M_Score
    FROM rfm_base
)
SELECT 
    CASE 
        WHEN R_Score = 4 AND F_Score = 4 AND M_Score = 4 THEN 'A. VIP / Champions'
        WHEN (R_Score >= 3 AND F_Score >= 3) OR M_Score = 4 THEN 'B. Loyal Customers'
        WHEN R_Score = 1 AND F_Score <= 2 THEN 'D. At Risk / Churning'
        WHEN R_Score >= 3 AND F_Score <= 2 THEN 'C. New Customers'
        ELSE 'E. Other'
    END as Segment,
    COUNT(CustomerID) as CustomerCount,
    ROUND(AVG(Monetary), 2) as AvgMonetary
FROM rfm_scores
GROUP BY Segment
ORDER BY CustomerCount DESC
""")
print("✅ View 'vw_customer_segments' creada (Corrección aplicada)")

## Generando graficos con las vistas

In [0]:
# ----------------------------------------------------------------------------
# GRÁFICO FINAL 1: Tendencia de Ingresos (Usando la Vista)
# ----------------------------------------------------------------------------
print("\n📊 DASHBOARD VIZ 1: Tendencia Temporal")

# Consultamos la vista optimizada
df_dash_1 = spark.sql("SELECT * FROM vw_monthly_revenue ORDER BY YearMonth")

display(df_dash_1)

Databricks visualization. Run in Databricks to view.

In [0]:
# ----------------------------------------------------------------------------
# GRÁFICO FINAL 2: Segmentación de Clientes (Usando la Vista)
# ----------------------------------------------------------------------------
print("\n📊 DASHBOARD VIZ 2: Distribución de Segmentos")

# Consultamos la vista optimizada
df_dash_2 = spark.sql("SELECT * FROM vw_customer_segments ORDER BY CustomerCount DESC")

display(df_dash_2)

Databricks visualization. Run in Databricks to view.

# 📊 PRESENTACION DEL DASHBOARD

---
# 🚀 CÓMO VER EL DASHBOARD EJECUTIVO

Este proyecto incluye un **Dashboard Interactivo** diseñado para la presentación de resultados.

**Para visualizarlo:**
1.  Por favor ir a la barra de herramientas superior del Notebook.
2.  Buscar la opción **View** (o el ícono de vista).
3.  Seleccionar **"Dashboard"**  **"EDA E-Commerce Executive Summary"**.
4.  Alternativamente, hacer clic en el botón **"View: Dashboard"** que puede aparecer en el menú lateral derecho.

---
**Nota:** El Dashboard consolida las métricas clave de RFM, Tendencias y Top Productos en una vista limpia sin código.